In [23]:
from datetime import datetime, timedelta
import calendar
import re

# 날짜 범위 설정

def parse_date_range_from_text(text: str):
    today = datetime.today()

    if re.search(r"오늘부터\s*일주일", text):
        start = today
        end = today + timedelta(days=6)

    elif re.search(r"이번\s*주말", text):
        weekday = today.weekday()
        saturday = today + timedelta(days=(5 - weekday))
        sunday = saturday + timedelta(days=1)
        start, end = saturday, sunday

    elif re.search(r"다음\s*주말", text):
        weekday = today.weekday()
        saturday = today + timedelta(days=(5 - weekday + 7))
        sunday = saturday + timedelta(days=1)
        start, end = saturday, sunday

    elif re.search(r"이번\s*달", text):
        start = today.replace(day=1)
        last_day = calendar.monthrange(today.year, today.month)[1]
        end = today.replace(day=last_day)

    elif re.search(r"다음\s*달", text):
        year = today.year + (today.month // 12)
        month = (today.month % 12) + 1
        start = datetime(year, month, 1)
        last_day = calendar.monthrange(year, month)[1]
        end = datetime(year, month, last_day)

    else:
        return None, None
    
    return start.strftime("%Y%m%d"), end.strftime("%Y%m%d")

In [24]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()

# 행사 정보 가져오기

def get_events(start_date, end_date, area_code=None, sigungu_code=None):
    api_key = os.getenv("KTO_API_KEY")  # .env에서 키 가져오기
    url = "https://apis.data.go.kr/B551011/KorService2/searchFestival2?MobileOS=ETC&MobileApp=Tourmate&eventStartDate=20250101&serviceKey=imXXm%2BLh3dotmxheNARbo8u5Xtuxb4XDPdquwpY0vgQFjs53Is7ofsYZLYly4YMKOjjlafy743m4FqBG%2F7ar5w%3D%3D&eventEndDate=20251231"

    params = {
        "serviceKey": api_key,
        "eventStartDate": start_date,
        "eventEndDate": end_date,
        "MobileOS": "ETC",
        "MobileApp": "TourChat",
        "areaCode": area_code,
        "numOfRows": 10,
        "_type": "json"
    }

    response = requests.get(url, params=params)
    data = response.json()
    return data.get("response", {}).get("body", {}).get("items", {}).get("item", [])

In [25]:
# 지역 코드 가져오기

def get_area_codes(region_name, subregion_name=None):
    import requests
    import os

    api_key = os.getenv("KTO_API_KEY")
    base_url = "https://apis.data.go.kr/B551011/KorService2/areaCode2?MobileOS=ETC&MobileApp=Tourmate&serviceKey=imXXm%2BLh3dotmxheNARbo8u5Xtuxb4XDPdquwpY0vgQFjs53Is7ofsYZLYly4YMKOjjlafy743m4FqBG%2F7ar5w%3D%3D"

    params = {
        "serviceKey": api_key,
        "MobileOS": "ETC",
        "MobileApp": "TourChat",
        "_type": "json"
    }

    region_code = None
    subregion_code = None

    r = requests.get(base_url, params=params)
    region_list = r.json()["response"]["body"]["items"]["item"]

    for region in region_list:
        if region_name in region["name"]:
            region_code = region["code"]
            break

    # Step 2: 시군구 코드 (옵션)
    if region_code and subregion_name:
        params["areaCode"] = region_code
        r2 = requests.get(base_url, params=params)
        subregion_list = r2.json()["response"]["body"]["items"]["item"]

        for sub in subregion_list:
            if subregion_name in sub["name"]:
                subregion_code = sub["code"]
                break

    return region_code, subregion_code

In [ ]:
def chatbot_query(text):
    # 날짜 자동 파싱
    start, end = parse_date_range_from_text(text)
    if not start or not end:
        return "날짜를 이해하지 못했어요."

    # 지역명 추출
    import re
    match = re.search(r"(서울|부산|대구|광주|제주|경기|전남|전북|경남|경북|충남|충북|강원|인천)(\s*(\S+))?", text)
    if match:
        region = match.group(1)
        subregion = match.group(3)
    else:
        region = "서울"  # 기본값
        subregion = None

    area_code, sigungu_code = get_area_codes(region, subregion)

    # 행사 요청
    events = get_events(start, end, area_code=str(area_code), sigungu_code=str(sigungu_code) if sigungu_code else None)

    if not events:
        return f"{region} 지역에서 {start} ~ {end} 기간에 열리는 행사가 없어요."

    reply = f"📍 {region} {subregion or ''} : ({start}~{end})\n"
    for event in events:
        reply += f"- {event.get('title')} ({event.get('addr1', '주소 정보 없음')})\n"
    return reply

In [27]:
print(chatbot_query("다음 주말 서울 행사 뭐 있어?"))

📍 서울 행사 : (20250712~20250713)
- 가락몰 빵축제 전국빵지자랑 (서울특별시 송파구 양재대로 932 (가락동))
- 강북구 어린이날 대축제 (서울특별시 강북구 월계로 173 (번동))
- 강서 아이들 까치까치 페스티벌 (서울특별시 강서구 우장산로 66 (내발산동))
- 강서구 다문화축제 (서울특별시 강서구 강서로5길 50 (화곡동))
- 강서봄빛페스타 (서울특별시 강서구 금낭화로 178 (방화동))
- 강서어린이 동화축제 (서울특별시 강서구 방화동)
- 강원관광두레 팝업스토어 (서울특별시 성동구 뚝섬로13길 36 (성수동2가))
- 겸재문화예술제 (서울특별시 강서구 양천로 291 (마곡동))
- 경복궁 별빛야행 (서울특별시 종로구 사직로 161 (세종로))
- 경복궁 생과방 (서울특별시 종로구 사직로 161 (세종로))

